In [1]:
interp.repositories() ++= Seq(coursierapi.MavenRepository.of(
"https://jitpack.io"
))

In [2]:
import $ivy. `org.carbonateresearch::picta:0.1`

import $ivy.$                                  

In [3]:
import picta.Html._ // required to initialize jupyter notebook mode
init_notebook_mode() // stops ugly output

import picta.Html._ // required to initialize jupyter notebook mode


# 1. Basics

The aim of the Picta library is to be a highly configurable and composable general purpose charting library. The following examples are aimed at highlighting the charting grammar for the library, which should make constructing charts easy and intuitive.

#### Colors

Colors in the library are represented as one of two types:

1. `List[String]` => this may either be a color word, such as "red", or RGB value, such as "rgb(255, 255, 255, 1)".
2. `List[Double]`

#### Lines

Lines are perhaps the simplest chart component. The constructor takes in the following arguments:

1. `width: Double`
2. `color: List[Double] or List[String] or String`

They may be composed as follows:

In [4]:
import picta.options.Line

val line = Line() + "rgb(255, 255, 255, 1)" // the default constructor for Line has a width of 0.5

println(line)

Line(0.5,Opt(Some(List(rgb(255, 255, 255, 1)))))


#### Markers

Markers take in the following optional parameters:

1. `symbol: String`
2. `color: List[String] or List[Double]`
3. `line: Line`

Again, composition of a line follows a similar pattern. However, we must wrap the color/s inside a `List` to avoid confusing the compiler. Here we create a `circle` marker, with the color `red` and a `line` of width 0.5:

In [5]:
import picta.options.Marker

val marker = Marker() + "circle" + List("red") + Line()

println(marker)

Marker(Opt(Some(circle)),Opt(Some(List(red))),Opt(Some(Line(0.5,Opt(None)))))


### [TO BE CONTINUED.....]

## Chart compositions

Charts consist of 3 components:

1. `data`
2. `layout`
3. `config`

Generally, most of the compositions make use of `data` and `layout` components.

#### Scatter Charts and Line Charts

In [6]:
// some dummy data for x and y variables
val x = List.range(1, 50)
val y = x.map(x => x + scala.util.Random.nextDouble()*100)

In [7]:
import picta.series.XY
import picta.series.XYChartType._
import picta.series.ModeType._
import picta.common.OptionWrapper._

// the two data series are then wrapped inside 
val data = XY(x, y, series_name="test", series_type=SCATTER, series_mode=MARKERS)

In [8]:
import picta.options.{Layout, Config}

// The layout is a composable object which determines how the chart is displayed
val layout = Layout(title="Chart")
// the configuration option allows us to set whether the chart is responsible and zoomable using scroll
val config = Config(responsive=false, scrollZoom=true)

In [9]:
import picta.charts.Chart

// we can compose a chart using the data, layout and config elements above. This grammar is the same for all charts
val chart = Chart() + data + layout + config
chart.plot_inline()

#### Adding Additional Traces to a Chart

We can compose an existing chart with more data series using the same composition technique as shown above for other components:

In [10]:
// additional traces can simply be composed with an existing chart and added on
val data1 = XY(x, y.map(x => x + scala.util.Random.nextDouble()*100), series_name="data1", series_type=BAR)

val chart1 = chart + data1
chart1.plot_inline()

#### Pie Chart

In [11]:
// pie charts can be composed in a similar way:
val data = XY(List(19, 26, 55), List("Residential", "Non-Residential", "Utility"), series_type=PIE)
val layout = Layout("Pie Chart")
val chart = Chart() + data +  layout + config
chart.plot_inline()

#### Setting Axes

For all charts, each axis can be composed invidiually. The following is an example for a `Histogram` chart:

In [12]:
val x = List.range(1, 1000).map(_+scala.util.Random.nextDouble()*1000)

// for histogram charts, the xkey must be specified. It essentially sets the orientation (x direction or y direction)
val data = XY(x=x, xkey="x", series_type=HISTOGRAM)

In [13]:
import picta.options.Axis

// axis can be set composed like any other chart component
val xaxis = Axis(key="xaxis", title = "my x data")
val yaxis = Axis(key="yaxis", title = "my y data")

// these are added to the layout object
val layout = Layout("Histogram with axes") + xaxis + yaxis

val chart = Chart() + data + layout  + config
chart.plot_inline()

#### Customizing colors for a Histogram

In [14]:
// we can also compose customizations in much the same way:
val marker = Marker() + List("rgba(255, 100, 102, 0.4)") + Line()

// change xkey to y to get a horizontal histogram
val data = XY(x, xkey="y", series_type=HISTOGRAM) + marker

val chart = Chart() + data + Layout("XY.Histogram.Color") + config
chart.plot_inline()

#### 2D Histogram Contour

In [15]:
val x = List.range(1, 50)
val y = x.map(x => x + scala.util.Random.nextDouble()*100)

val data = XY(x, y, series_type=HISTOGRAM2DCONTOUR, series_mode=MARKERS)

val chart = Chart() + data + Layout("2D Histogram Contour") + config
chart.plot_inline()

#### Contour

This is a 3 dimensional chart requiring the import of the `XYZ` class from the library.

In [16]:
import picta.series.XYZ
import picta.series.XYZChartType._

val x = List.range(1, 100)
val y = List.range(1, 100)
val z = List.range(1, 100).map(x => x + scala.util.Random.nextDouble()*100)

val data = XYZ(x, y, z, series_type=CONTOUR, series_mode=MARKERS)

In [17]:
val chart = Chart() + data + Layout("Contour")
chart.plot_inline()

#### Heatmap

In [18]:
val z = List.range(1, 101).map(e => e + scala.util.Random.nextDouble()*100).grouped(10).toList

In [19]:
// add lines in between the grid items
val data = XYZ(z, series_type=HEATMAP)

val chart = Chart() + data + Layout("Contour", height = 500, width = 500)
chart.plot_inline()

#### Scatter3D

In [20]:
val x = List.range(1, 100)
val y = List.range(1, 100)
val z = List.range(1, 100).map(e => e + scala.util.Random.nextDouble()*100)

val data = XYZ(x, y, z, series_type = SCATTER3D)

val chart = Chart() + data + layout + config
chart.plot_inline()

#### Surface Plot

In [21]:
// 3d surface plot
val k = List(
    List(8.83,8.89,8.81,8.87,8.9,8.87),
    List(8.89,8.94,8.85,8.94,8.96,8.92),
    List(8.84,8.9,8.82,8.92,8.93,8.91),
    List(8.79,8.85,8.79,8.9,8.94,8.92),
    List(8.79,8.88,8.81,8.9,8.95,8.92),
    List(8.8,8.82,8.78,8.91,8.94,8.92),
    List(8.75,8.78,8.77,8.91,8.95,8.92),
    List(8.8,8.8,8.77,8.91,8.95,8.94),
    List(8.74,8.81,8.76,8.93,8.98,8.99),
    List(8.89,8.99,8.92,9.1,9.13,9.11),
    List(8.97,8.97,8.91,9.09,9.11,9.11),
    List(9.04,9.08,9.05,9.25,9.28,9.27),
    List(9,9.01,9,9.2,9.23,9.2),
    List(8.99,8.99,8.98,9.18,9.2,9.19),
    List(8.93,8.97,8.97,9.18,9.2,9.18)
  )


val data = XYZ(k, series_name="surface", series_type=SURFACE)

val chart = Chart() + data + Layout("Surface Chart")
chart.plot_inline()

#### Third Dimension as Color

In [22]:
// multiple compositions can be used to create scatter charts with a color representing some third dimension

val marker = Marker() + z // we add the list of colors to the marker for each point
val data = XY(x, y, series_name="scatter with color", series_type = SCATTER, series_mode = MARKERS) + marker
val chart = Chart() + Layout("Color2D.Basic") + config + data
chart.plot_inline()

#### Grid

The grid system can be used to generate subplots:

In [23]:
import picta.options.Grid

// 1. first we define the grid layout - 1 row, 2 columns
val grid = Grid(1, 2) // rename to subplot

// 2. Now define the axes we want to place on the grid 
val ax1 = Axis("xaxis", title = "x axis 1")
val ax2 = Axis("xaxis2", title = "x axis 2")

// 3. define the traces - notice how the axis maps to the xaxis and yaxis keys below:
val data1 = XY(x, y, series_name="a", series_type=SCATTER, series_mode=MARKERS)
val data2 = XY(x, z, series_name="b", series_type=SCATTER, series_mode=MARKERS, xaxis="x2", yaxis="y2")


// 4. combine the axis and grid into a single layout component
val layout = Layout("Axis Composition") + List(ax1, ax2) + grid

// 5. construct into a chart. We do not need to put the series into a single list, but this is done as an example here
val chart = Chart() + List(data1, data2) + layout + config

chart.plot_inline()

#### Map

We can also create maps using the composition technique below.

In [24]:
import picta.options.Geo
import picta.options.MapAxis
import picta.options.LatAxis
import picta.options.LongAxis
import picta.series.Map

val line = Line(width = 2, color=List("red"))
val data = Map(lat=List(40.7127, 51.5072), lon=List(-74.0059, 0.1275), series_mode = LINES) + line


val geo = Geo(landcolor = "rgb(204, 204, 204)", lakecolor="rgb(255, 255, 255)") + LatAxis(List(20, 60)) + 
LongAxis(List(-100, 20))

val layout = Layout() + geo
val chart = Chart() + data + layout
chart.plot_inline()

#### Animated Chart

In [25]:
import picta.series.{XYSeries, XYZSeries}


// creates random XY for testing purposes
def createXYSeries(numberToCreate: Int, count: Int = 0, length: Int = 10): List[XYSeries] = {
    if (count == numberToCreate) Nil
    else {
      val xs = List.range(0, length)
      val ys = xs.map(x => scala.util.Random.nextDouble() * x)
      val trace = XY(x=xs, y = ys, series_name = "trace" + count)
      trace :: createXYSeries(numberToCreate, count + 1, length)
    }
}

def createXYZSeries(numberToCreate: Int, count: Int = 0, length: Int = 10): List[XYZSeries] = {
    if (count == numberToCreate) Nil
    else {
      val xs = List.range(0, length)
      val ys = xs.map(x => scala.util.Random.nextDouble() * x)
      val zs = xs.map(x => scala.util.Random.nextDouble() * x * scala.util.Random.nextInt())
      val trace = XYZ(x=xs, y=ys, z=zs, series_name = "trace" + count, series_type = SCATTER3D)
      trace :: createXYZSeries(numberToCreate, count + 1, length)
    }
}

In [26]:
import picta.options.animation._

val xaxis = Axis(key="xaxis", title="X Variable", range = (0.0, 10.0))
val yaxis = Axis(key="yaxis", title="Y Variable", range = (0.0, 10.0))
val layout = Layout("Animation.XY")  + xaxis + yaxis + AnimationEngine()
val traces = createXYSeries(numberToCreate = 100)
val chart = Chart(animated=true) + layout + traces

chart.plot_inline()


In [27]:
val series = createXYZSeries(numberToCreate=100)
val layout = Layout("Animation.XYZ") + AnimationEngine()
val chart = Chart(animated=true) + layout + series

chart.plot_inline()